This notebook introduces various metrics that will be used for product performance analysis and evaluation. Below are the key metrics that will be discussed:


### Total Sales (`total_sales`):
- **Definition:** The sum of the 'sales' column across the dataset.
- **Purpose:** Provides a baseline to understand the overall revenue generated from the products.

### Total Profit (`total_profit`):
- **Definition:** The sum of the 'profit' column across the dataset.
- **Purpose:** Indicates the total earnings.

### Total Units Sold (`total_units_sold`):
- **Definition:** Sum of the 'quantity' sold per product.
- **Purpose:** Measures the volume of products sold, useful for sales performance analysis.

### Average Sale Price (`average_sale_price`):
- **Definition:** Average of the 'sales' values per product.
- **Purpose:** Helps understand the typical selling price per order, useful for pricing strategies.

### Average Discount (`average_discount`):
- **Definition:** Average of the 'discount' provided per product.
- **Purpose:** Indicates how much discount is typically given, which can affect profitability and sales strategies.

### Average Processing Time (`average_processing_time`):
- **Definition:** Average processing time per product.
- **Purpose:** Useful for assessing operational efficiency and customer service quality.

### Total Returns (`total_returns`):
- **Definition:** Sum of negative quantities for each product group, indicating returned items.
- **Purpose:** Highlights product return rates which can inform quality issues or customer satisfaction problems.

### Total Loss (`total_loss`):
- **Definition:** Sum of negative profit values, indicating financial losses from returns or other factors.
- **Purpose:** Identifies loss-making aspects of the product portfolio.

### Profit Margin (`profit_margin`):
- **Definition:** Calculated as (Total Profit / Total Sales) * 100 for each product.
- **Purpose:** Shows the profitability percentage, indicating how much of the sales convert into profit.

### Average Profit Per Unit (`average_profit_per_unit`):
- **Definition:** Calculated by dividing total profit by total units sold per product.
- **Purpose:** Assesses how much profit is made per unit sold, helping in evaluating product performance.

### Sales Percentage of Total (`sales_percentage_of_total`):
- **Definition:** (Total Sales of a product / Total Sales of all products) * 100.
- **Purpose:** Indicates the contribution of each product's sales to the overall sales, useful for identifying key revenue drivers.

### Profit Rank (`profit_rank`) and Sales Rank (`sales_rank`):
- **Definition:** These ranks are calculated based on the descending order of total profit and total sales, respectively.
- **Purpose:** Helps in quickly identifying top-performing products in terms of profitability and sales volume.

### Discount Impact on Sales (`discount_impact_on_sales`):
- **Definition:** Calculated as the percentage change in total sales for each product relative to the average sales of products with the lowest 10% of discounts.
- **Purpose:** Measures how variations in discount levels influence sales compared to a baseline, providing insights into the effectiveness of discount strategies.

### Discount Impact on Profit (`discount_impact_on_profit`):
- **Definition:** Calculated as the percentage change in total profit for each product relative to the average profit of products with the lowest 10% of discounts.
- **Purpose:** Evaluates the impact of discounts on profitability, helping to assess whether discount strategies are enhancing or harming overall financial performance.

### Active Months (`active_months`):
- **Definition:** The count of unique year-month combinations during which each product was sold, derived from the order dates.
- **Purpose:** Indicates the duration of active sales for each product, providing a measure of product lifespan and market presence.

### Composite Score (`composite_score`):
- **Definition:** A combined score derived by summing the normalized values of sales, adjusted profit, profit margin, and active months for each product.
- **Purpose:** Provides a single, holistic measure of product performance that incorporates financial success, pricing efficiency, market engagement, and profitability adjustments, aiding in comprehensive product evaluation and decision-making.

These metrics serve different purposes and collectively contribute to a comprehensive analysis of product performance and financial success.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Set option to display all columns of the DataFrame
pd.set_option('display.max_columns', None)

In [3]:
data_path = '../data/cleaned_ecommerce.csv'

try:
    data = pd.read_csv(data_path, encoding='utf-8')
except UnicodeDecodeError:
    data = pd.read_csv(data_path, encoding='ISO-8859-1')  # Using ISO-8859-1 if UTF-8 fails


In [4]:
data

,order_id,date_ordered,date_shipped,ship_mode,customer_id,segment,city,state,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit,processing_time
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,3
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,3
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,4
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,7
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9933,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Consumer,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,2
9934,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Consumer,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,5
9935,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Consumer,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,5
9936,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Consumer,Costa Mesa,California,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,5


### 1. Standardize on the Most Common Product Name

In [5]:
# Group by 'product_id' and 'product_name', and count occurrences
name_counts = data.groupby(['product_id', 'product_name']).size().reset_index(name='count')

# Sort the results and keep the most common name for each product_id
most_common_names = name_counts.sort_values(by=['product_id', 'count'], ascending=[True, False]).drop_duplicates('product_id')

# Merge this back to the original data to standardize the product names
standardized_data = data.merge(most_common_names[['product_id', 'product_name']], on='product_id', suffixes=('', '_standardized'))

# Use the standardized product name for further analysis
standardized_data['product_name'] = standardized_data['product_name_standardized']
standardized_data.drop(columns=['product_name_standardized'], inplace=True)

standardized_data

,order_id,date_ordered,date_shipped,ship_mode,customer_id,segment,city,state,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit,processing_time
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2,0.00,41.9136,3
1,CA-2016-117681,2016-04-09,2016-04-14,Standard Class,HF-14995,Consumer,Los Angeles,California,90004,West,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,556.665,5,0.15,6.5490,5
2,CA-2017-159793,2017-08-25,2017-08-29,Standard Class,SV-20365,Consumer,Philadelphia,Pennsylvania,19140,East,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,130.980,2,0.50,-89.0664,4
3,CA-2017-110198,2017-05-01,2017-05-02,First Class,AG-10900,Consumer,Coral Springs,Florida,33065,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,314.352,3,0.20,-15.7176,1
4,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940,3,0.00,219.5820,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9933,CA-2014-169446,2014-12-19,2014-12-25,Standard Class,SG-20605,Consumer,Chicago,Illinois,60623,Central,TEC-PH-10002817,Technology,Phones,RCA ViSYS 25425RE1 Corded phone,323.976,3,0.20,36.4473,6
9934,CA-2016-129280,2016-05-03,2016-05-05,First Class,SM-20905,Consumer,Newark,Ohio,43055,East,TEC-MA-10003589,Technology,Machines,Cisco 8961 IP Phone Charcoal,224.937,3,0.70,-164.9538,2
9935,CA-2015-138625,2015-11-02,2015-11-05,First Class,EG-13900,Consumer,Chesapeake,Virginia,23320,South,OFF-AP-10003099,Office Supplies,Appliances,"Eureka Hand Vacuum, Bagless",197.720,4,0.00,55.3616,3
9936,CA-2016-114867,2016-12-23,2016-12-28,Standard Class,FM-14290,Home Office,Philadelphia,Pennsylvania,19143,East,TEC-PH-10002645,Technology,Phones,LG G2,1499.970,5,0.40,-374.9925,5


In [6]:
data = standardized_data

In [7]:
# Calculate the sale price per unit for each transaction
data['sale_price_per_unit'] = data['sales'] / data['quantity']

# Calculate total metrics for perspective
total_sales = data['sales'].sum()
total_profit = data['profit'].sum()

# Group data by 'product_id' and 'product_name' to calculate the desired metrics
product_metrics = data.dropna().groupby(['product_id', 'product_name', 'category', 'subcategory']).agg(
    total_sales=('sales', 'sum'),
    total_units_sold=('quantity', 'sum'),
    average_sale_price=('sales', 'mean'),
    min_sale_price_per_unit=('sale_price_per_unit', 'min'),  # Minimum sale price per unit
    max_sale_price_per_unit=('sale_price_per_unit', 'max'),  # Maximum sale price per unit
    avg_sale_price_per_unit=('sale_price_per_unit', 'mean'),  # Mean sale price per unit
    total_profit=('profit', 'sum'),
    average_discount=('discount', 'mean'),
    average_processing_time=('processing_time', 'mean'),
    total_returns=('quantity', lambda x: x[x < 0].sum()),  # Summing only negative quantities
    total_loss=('profit', lambda x: x[x < 0].sum())  # Summing only negative profit
).reset_index()

# Calculate Profit Margin as a percentage
product_metrics['profit_margin'] = product_metrics['total_profit'] / product_metrics['total_sales'] * 100

# Recalculate Total Orders and merge correctly with the product metrics
order_counts = data.groupby('product_id')['order_id'].nunique().reset_index()
product_metrics = product_metrics.merge(order_counts, on='product_id', how='left')
product_metrics.rename(columns={'order_id': 'total_orders'}, inplace=True)

# Additional KPIs
product_metrics['average_profit_per_unit'] = product_metrics['total_profit'] / product_metrics['total_units_sold']
product_metrics['sales_percentage_of_total'] = product_metrics['total_sales'] / total_sales * 100
product_metrics['profit_rank'] = product_metrics['total_profit'].rank(ascending=False)
product_metrics['sales_rank'] = product_metrics['total_sales'].rank(ascending=False)

product_metrics

,product_id,product_name,category,subcategory,total_sales,total_units_sold,average_sale_price,min_sale_price_per_unit,max_sale_price_per_unit,avg_sale_price_per_unit,total_profit,average_discount,average_processing_time,total_returns,total_loss,profit_margin,total_orders,average_profit_per_unit,sales_percentage_of_total,profit_rank,sales_rank
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",Furniture,Bookcases,825.174,9,825.174000,91.686,91.686,91.686000,-117.8820,0.300000,2.000000,0,-117.8820,-14.285714,1,-13.098000,0.036223,1740.0,570.0
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",Furniture,Bookcases,1064.624,10,354.874667,102.833,120.980,108.882000,24.1960,0.100000,1.666667,0,-9.6784,2.272727,3,2.419600,0.046735,1152.0,499.0
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases,2154.348,14,430.869600,119.686,170.980,148.752600,311.1836,0.130000,4.000000,0,-35.9058,14.444444,5,22.227400,0.094571,268.0,283.0
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,Furniture,Bookcases,723.842,21,120.640333,14.574,48.580,39.673667,-133.1092,0.183333,3.666667,0,-195.2916,-18.389262,6,-6.338533,0.031775,1751.0,602.0
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",Furniture,Bookcases,851.760,12,425.880000,70.980,70.980,70.980000,229.9752,0.000000,6.500000,0,0.0000,27.000000,2,19.164600,0.037391,337.0,560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,Technology,Phones,516.530,11,172.176667,43.960,54.950,47.623333,87.3705,0.133333,3.333333,0,0.0000,16.914894,3,7.942773,0.022675,639.0,723.0
1856,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,1085.238,19,217.047600,53.592,66.990,58.951200,130.6305,0.120000,4.000000,0,0.0000,12.037037,5,6.875289,0.047640,481.0,491.0
1857,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",Technology,Phones,56.164,8,18.721333,5.912,7.390,6.897333,13.0064,0.066667,2.333333,0,0.0000,23.157895,3,1.625800,0.002465,1351.0,1603.0
1858,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,Technology,Phones,341.666,4,170.833000,80.392,100.490,90.441000,40.1960,0.100000,3.000000,0,0.0000,11.764706,2,10.049000,0.014998,974.0,882.0


In [8]:
product_metrics[product_metrics['total_returns']!=0]

,product_id,product_name,category,subcategory,total_sales,total_units_sold,average_sale_price,min_sale_price_per_unit,max_sale_price_per_unit,avg_sale_price_per_unit,total_profit,average_discount,average_processing_time,total_returns,total_loss,profit_margin,total_orders,average_profit_per_unit,sales_percentage_of_total,profit_rank,sales_rank
323,FUR-TA-10001039,KI Adjustable-Height Table,Furniture,Tables,2764.257,-47,251.296091,-85.98,-42.99,-66.829909,-226.1274,0.222727,3.636364,-47,-494.3850,-8.180404,11,4.811221,0.121345,1780.0,219.0
361,FUR-TA-10004086,KI Adjustable-Height Table,Furniture,Tables,1788.384,-27,255.483429,-85.98,-42.99,-63.870857,-22.3548,0.257143,4.285714,-27,-216.6696,-1.250000,7,0.827956,0.078506,1666.0,328.0


In [9]:
data[data['product_id']=='FUR-TA-10001039']

,order_id,date_ordered,date_shipped,ship_mode,customer_id,segment,city,state,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit,processing_time,sale_price_per_unit
6001,US-2014-118486,2014-04-06,2014-04-08,First Class,SD-20485,Home Office,Philadelphia,Pennsylvania,19143,East,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,154.764,-3,0.40,-36.1116,2,-51.588
6002,CA-2015-112116,2015-03-16,2015-03-18,Second Class,JE-15475,Consumer,Seattle,Washington,98103,West,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,171.960,-2,0.00,44.7096,2,-85.980
6003,CA-2014-104283,2014-06-27,2014-07-01,Standard Class,LM-17065,Consumer,Southaven,Mississippi,38671,South,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,85.980,-1,0.00,22.3548,4,-85.980
6004,CA-2017-148404,2017-10-07,2017-10-11,Standard Class,Dp-13240,Home Office,Charlotte,North Carolina,28205,South,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,154.764,-3,0.40,-36.1116,4,-51.588
6005,CA-2014-160766,2014-09-14,2014-09-14,Same Day,DM-13015,Consumer,New York City,New York,10009,East,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,464.292,-9,0.40,-108.3348,0,-51.588
6006,CA-2014-117765,2014-09-07,2014-09-13,Standard Class,RB-19465,Home Office,Tulsa,Oklahoma,74133,Central,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,429.900,-5,0.00,111.7740,6,-85.980
6007,CA-2017-135937,2017-02-20,2017-02-27,Standard Class,KM-16375,Home Office,Gilbert,Arizona,85234,West,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,386.910,-9,0.50,-185.7168,7,-42.990
6008,CA-2017-141439,2017-11-26,2017-12-01,Standard Class,TT-21460,Home Office,Richmond,Indiana,47374,Central,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,257.940,-3,0.00,67.0644,5,-85.980
6009,CA-2016-104276,2016-11-27,2016-12-03,Standard Class,HF-14995,Consumer,Fort Lauderdale,Florida,33311,South,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,331.023,-7,0.45,-114.3534,6,-47.289
6010,CA-2017-125451,2017-10-23,2017-10-24,First Class,AH-10075,Corporate,Cranston,Rhode Island,2920,East,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,240.744,-4,0.30,-13.7568,1,-60.186


In [10]:
data[data['order_id']=='CA-2016-104276']

,order_id,date_ordered,date_shipped,ship_mode,customer_id,segment,city,state,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit,processing_time,sale_price_per_unit
6009,CA-2016-104276,2016-11-27,2016-12-03,Standard Class,HF-14995,Consumer,Fort Lauderdale,Florida,33311,South,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,331.023,-7,0.45,-114.3534,6,-47.289
7670,CA-2016-104276,2016-11-27,2016-12-03,Standard Class,HF-14995,Consumer,Fort Lauderdale,Florida,33311,South,TEC-PH-10001944,Technology,Phones,Wi-Ex zBoost YX540 Cellular Phone Signal Booster,116.760,1,0.20,14.5950,6,116.760


In [11]:
# Calculate correlation between discount and sales
discount_sales_correlation = np.corrcoef(product_metrics['average_discount'], product_metrics['total_sales'])[0, 1]

# Calculate correlation between discount and profitability
discount_profit_correlation = np.corrcoef(product_metrics['average_discount'], product_metrics['profit_margin'])[0, 1]

print("Discount and Sales Correlation:", discount_sales_correlation)
print("Discount and Profitability Correlation:", discount_profit_correlation)

Discount and Sales Correlation: 0.07077440070375345
Discount and Profitability Correlation: -0.6651633050209477


**1. Impact on Sales:**

**Observation**: There is a very weak positive correlation between discounts and sales volumes (Correlation Coefficient: 0.073).
**Implication**: Based on the available data, discounts have minimal impact on increasing the sales volumes. This suggests that customers' purchasing decisions might not be heavily influenced by price reductions.
**Recommendation**: Consider exploring alternative sales promotion tactics beyond discounts, such as improving products, enhancing customer service, or investing in marketing campaigns that highlight the unique selling propositions.

**2. Impact on Profitability:**

**Observation:** There is a moderate to strong negative correlation between discounts and profitability (Correlation Coefficient: **-0.664**).
**Implication:** Based on the available data, higher discounts significantly reduce the profit margins. This underscores the cost of discounts as a sales strategy.
**Recommendation:** Reevaluate our discount strategy to focus on more targeted, strategic discounting that maintains or improves profitability. This might include selective discounts for high-margin products or bundled offerings that encourage larger purchase sizes without eroding profit margins.

**Conclusion:**
While discounts are a common tactic to attract customers, the data suggests that their effectiveness in boosting sales is limited, and they pose a risk to the profitability. A shift towards more strategic discounting, coupled with alternative promotional activities, could better serve the long-term growth and profitability goals.

In [12]:
# Assuming a minimal baseline discount for comparison (e.g., the lowest 10% of discounts)
baseline_discount = product_metrics['average_discount'].quantile(0.1)
baseline_sales = product_metrics[product_metrics['average_discount'] <= baseline_discount]['total_sales'].mean()
baseline_profit = product_metrics[product_metrics['average_discount'] <= baseline_discount]['total_profit'].mean()

# Calculate discount impact on sales and profit as percentage change from the baseline
product_metrics['discount_impact_on_sales'] = (product_metrics['total_sales'] - baseline_sales) / baseline_sales * 100
product_metrics['discount_impact_on_profit'] = (product_metrics['total_profit'] - baseline_profit) / baseline_profit * 100

product_metrics[['product_id', 'product_name', 'average_discount', 'total_sales', 'discount_impact_on_sales','total_profit', 'discount_impact_on_profit']]


,product_id,product_name,average_discount,total_sales,discount_impact_on_sales,total_profit,discount_impact_on_profit
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",0.300000,825.174,35.841523,-117.8820,-156.782664
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",0.100000,1064.624,75.260182,24.1960,-88.345012
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,0.130000,2154.348,254.652369,311.1836,49.894247
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,0.183333,723.842,19.160080,-133.1092,-164.117464
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",0.000000,851.760,40.218155,229.9752,10.776916
...,...,...,...,...,...,...,...
1855,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,0.133333,516.530,-14.967968,87.3705,-57.914442
1856,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,0.120000,1085.238,78.653694,130.6305,-37.076502
1857,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",0.066667,56.164,-90.754188,13.0064,-93.734938
1858,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,0.100000,341.666,-43.754372,40.1960,-80.637960


In [13]:
data['date_ordered'] = pd.to_datetime(data['date_ordered'])

# Extract year-month from the date_ordered and store in a new column
data['year_month'] = data['date_ordered'].dt.to_period('M')

# Group by 'product_id' and count unique year-month combinations
product_active_months = data.groupby('product_id')['year_month'].nunique().reset_index()

# Rename the column appropriately
product_active_months.rename(columns={'year_month': 'active_months'}, inplace=True)

# Merge this back into your original metrics DataFrame
product_metrics = product_metrics.merge(product_active_months, on='product_id', how='left')

# Show the DataFrame to verify
product_metrics


,product_id,product_name,category,subcategory,total_sales,total_units_sold,average_sale_price,min_sale_price_per_unit,max_sale_price_per_unit,avg_sale_price_per_unit,total_profit,average_discount,average_processing_time,total_returns,total_loss,profit_margin,total_orders,average_profit_per_unit,sales_percentage_of_total,profit_rank,sales_rank,discount_impact_on_sales,discount_impact_on_profit,active_months
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",Furniture,Bookcases,825.174,9,825.174000,91.686,91.686,91.686000,-117.8820,0.300000,2.000000,0,-117.8820,-14.285714,1,-13.098000,0.036223,1740.0,570.0,35.841523,-156.782664,1
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",Furniture,Bookcases,1064.624,10,354.874667,102.833,120.980,108.882000,24.1960,0.100000,1.666667,0,-9.6784,2.272727,3,2.419600,0.046735,1152.0,499.0,75.260182,-88.345012,3
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases,2154.348,14,430.869600,119.686,170.980,148.752600,311.1836,0.130000,4.000000,0,-35.9058,14.444444,5,22.227400,0.094571,268.0,283.0,254.652369,49.894247,5
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,Furniture,Bookcases,723.842,21,120.640333,14.574,48.580,39.673667,-133.1092,0.183333,3.666667,0,-195.2916,-18.389262,6,-6.338533,0.031775,1751.0,602.0,19.160080,-164.117464,6
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",Furniture,Bookcases,851.760,12,425.880000,70.980,70.980,70.980000,229.9752,0.000000,6.500000,0,0.0000,27.000000,2,19.164600,0.037391,337.0,560.0,40.218155,10.776916,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,Technology,Phones,516.530,11,172.176667,43.960,54.950,47.623333,87.3705,0.133333,3.333333,0,0.0000,16.914894,3,7.942773,0.022675,639.0,723.0,-14.967968,-57.914442,3
1856,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,1085.238,19,217.047600,53.592,66.990,58.951200,130.6305,0.120000,4.000000,0,0.0000,12.037037,5,6.875289,0.047640,481.0,491.0,78.653694,-37.076502,5
1857,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",Technology,Phones,56.164,8,18.721333,5.912,7.390,6.897333,13.0064,0.066667,2.333333,0,0.0000,23.157895,3,1.625800,0.002465,1351.0,1603.0,-90.754188,-93.734938,3
1858,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,Technology,Phones,341.666,4,170.833000,80.392,100.490,90.441000,40.1960,0.100000,3.000000,0,0.0000,11.764706,2,10.049000,0.014998,974.0,882.0,-43.754372,-80.637960,2


In [14]:
# Summarize the top-selling products by total sales and total units sold
top_sales = product_metrics.nlargest(10, 'total_sales')[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'total_units_sold', 'active_months']]
top_units_sold = product_metrics.nlargest(10, 'total_units_sold')[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'total_units_sold', 'active_months']]
top_profits = product_metrics.nlargest(10, 'total_profit')[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'total_units_sold', 'active_months']]



### Top-Selling Products by Total Sales

In [15]:
top_sales

,product_name,category,subcategory,total_sales,total_profit,total_units_sold,active_months
1614,Canon imageCLASS 2200 Advanced Copier,Technology,Copiers,61599.824,2.519993e+04,20,5
776,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,Binders,27453.384,7.753039e+03,31,10
1641,Cisco TelePresence System EX90 Videoconferenci...,Technology,Machines,22638.480,-1.811078e+03,6,1
80,HON 5400 Series Task Chairs for Big and Tall,Furniture,Chairs,21870.576,5.684342e-14,39,8
691,GBC DocuBind TL300 Electric Binding System,Office Supplies,Binders,19823.479,2.233505e+03,37,9
657,GBC Ibimaster 500 Manual ProClick Binding System,Office Supplies,Binders,19024.500,7.609800e+02,48,9
1604,Hewlett Packard LaserJet 3310 Copier,Technology,Copiers,18839.686,6.983884e+03,38,7
1630,HP Designjet T520 Inkjet Large Format Printer ...,Technology,Machines,18374.895,4.094977e+03,12,3
845,GBC DocuBind P400 Electric Binding System,Office Supplies,Binders,17965.068,-1.878166e+03,27,6
1420,High Speed Automatic Electric Letter Opener,Office Supplies,Supplies,17030.312,-2.620048e+02,11,2


### Top-Selling Products by Units Sold

In [16]:
top_units_sold

,product_name,category,subcategory,total_sales,total_profit,total_units_sold,active_months
1569,Imation 16GB Mini TravelDrive USB 2.0 Flash Drive,Technology,Accessories,11203.756,1987.9253,75,13
1144,Xerox 1881,Office Supplies,Paper,2167.348,1006.4352,70,15
694,GBC Premium Transparent Covers with Diagonal L...,Office Supplies,Binders,780.456,49.5128,67,11
721,Avery Arch Ring Binders,Office Supplies,Binders,2068.822,349.6332,64,10
93,"Situations Contoured Folding Chairs, 4/Set",Furniture,Chairs,3499.314,198.7440,62,13
325,Chromcraft Round Conference Tables,Furniture,Tables,8209.059,-189.9761,61,11
1517,Logitech G19 Programmable Gaming Keyboard,Technology,Accessories,13756.536,4425.3432,60,12
183,DAX Wood Document Frame,Furniture,Furnishings,839.676,176.3247,57,13
1548,Kingston Digital DataTraveler 16GB USB 2.0,Technology,Accessories,477.930,64.7085,57,12
1354,Fellowes Officeware Wire Shelving,Office Supplies,Storage,4509.466,-332.3710,55,10


### Top-Selling Products by Profit

In [17]:
top_profits

,product_name,category,subcategory,total_sales,total_profit,total_units_sold,active_months
1614,Canon imageCLASS 2200 Advanced Copier,Technology,Copiers,61599.824,25199.9280,20,5
776,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,Binders,27453.384,7753.0390,31,10
1604,Hewlett Packard LaserJet 3310 Copier,Technology,Copiers,18839.686,6983.8836,38,7
1611,Canon PC1060 Personal Laser Copier,Technology,Copiers,11619.834,4570.9347,19,4
1517,Logitech G19 Programmable Gaming Keyboard,Technology,Accessories,13756.536,4425.3432,60,12
1630,HP Designjet T520 Inkjet Large Format Printer ...,Technology,Machines,18374.895,4094.9766,12,3
1665,Ativa V4110MDD Micro-Cut Shredder,Technology,Machines,7699.890,3772.9461,11,2
1629,"3D Systems Cube Printer, 2nd Generation, Magenta",Technology,Machines,14299.890,3717.9714,11,2
682,Ibico EPK-21 Electric Binding System,Office Supplies,Binders,15875.916,3345.2823,13,3
1616,Zebra ZM400 Thermal Label Printer,Technology,Machines,6965.700,3343.5360,6,1


### Composite Score = (Normalized Total Sales) + (Normalized Total Profit) + (Normalized Profit Margin) + (Normalized Active Months)

In [18]:
# Selecting the relevant metrics
metrics = product_metrics[['product_id', 'product_name', 'total_sales', 'total_profit', 'profit_margin', 'active_months', 'category', 'subcategory']]


# Finding the minimum profit to adjust negative values
min_profit = product_metrics['total_profit'].min()


# Offset negative profits by adding the absolute value of the minimum profit + 1 to ensure all values are positive
profit_adjusted = product_metrics['total_profit'] + abs(min_profit) + 1

# Incorporate this adjusted profit into the normalized dataframe
metrics_adjusted = product_metrics[['total_sales', 'profit_margin', 'active_months']].copy()
metrics_adjusted['adjusted_profit'] = profit_adjusted

# Normalize the new metrics
scaler_adjusted = MinMaxScaler()
metrics_normalized_adjusted = scaler_adjusted.fit_transform(metrics_adjusted)
metrics_normalized_adjusted_df = pd.DataFrame(metrics_normalized_adjusted, columns=['normalized_sales', 'normalized_margin', 'normalized_adjusted_profit', 'normalized_active_months'])

# Calculate a new composite score with the adjusted profit
metrics_adjusted_combined = pd.concat([metrics, metrics_normalized_adjusted_df], axis=1)
metrics_adjusted_combined['composite_score'] = metrics_adjusted_combined['normalized_sales'] + metrics_adjusted_combined['normalized_adjusted_profit']+ metrics_adjusted_combined['normalized_margin']+ metrics_adjusted_combined['normalized_active_months']

# Merge the composite score back into the original DataFrame
product_metrics = product_metrics.merge(metrics_adjusted_combined[['product_id', 'composite_score']], on='product_id', how='left')

# Display the top products based on the new adjusted composite score
top_adjusted_composite_score_products = metrics_adjusted_combined.nlargest(10, 'composite_score')[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'profit_margin', 'active_months', 'composite_score']]
top_adjusted_composite_score_products


,product_name,category,subcategory,total_sales,total_profit,profit_margin,active_months,composite_score
1614,Canon imageCLASS 2200 Advanced Copier,Technology,Copiers,61599.824,25199.9280,40.909091,5,3.257742
776,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,Binders,27453.384,7753.0390,28.240741,10,2.509623
1517,Logitech G19 Programmable Gaming Keyboard,Technology,Accessories,13756.536,4425.3432,32.169023,12,2.344566
1144,Xerox 1881,Office Supplies,Paper,2167.348,1006.4352,46.436253,15,2.314288
1569,Imation 16GB Mini TravelDrive USB 2.0 Flash Drive,Technology,Accessories,11203.756,1987.9253,17.743383,13,2.258644
295,"Eldon Image Series Desk Accessories, Burgundy",Furniture,Furnishings,426.560,116.7148,27.361872,15,2.201231
1168,Adams Telephone Message Book W/Dividers/Space ...,Office Supplies,Paper,1004.652,415.2840,41.336104,14,2.190946
1604,Hewlett Packard LaserJet 3310 Copier,Technology,Copiers,18839.686,6983.8836,37.070064,7,2.160099
831,Ibico Hi-Tech Manual Binding System,Office Supplies,Binders,5535.593,-246.0675,-4.445188,14,2.104231
1547,Plantronics CS510 - Over-the-Head monaural Wir...,Technology,Accessories,10822.360,3085.0325,28.506098,10,2.103475


In [19]:
# Group data by category and display the top products based on the new adjusted composite score
top_products_by_category = metrics_adjusted_combined.groupby('category').apply(lambda x: x.nlargest(10, 'composite_score')).reset_index(drop=True)
top_products_by_category[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'profit_margin', 'active_months', 'composite_score']]

,product_name,category,subcategory,total_sales,total_profit,profit_margin,active_months,composite_score
0,"Eldon Image Series Desk Accessories, Burgundy",Furniture,Furnishings,426.5600,1.167148e+02,2.736187e+01,15,2.201231
1,DAX Wood Document Frame,Furniture,Furnishings,839.6760,1.763247e+02,2.099914e+01,13,2.047252
2,"Situations Contoured Folding Chairs, 4/Set",Furniture,Chairs,3499.3140,1.987440e+02,5.679513e+00,13,2.043949
3,"Global Value Mid-Back Manager's Chair, Gray",Furniture,Chairs,2442.0240,1.174863e+02,4.811022e+00,13,2.021729
4,Bretford Rectangular Conference Table Tops,Furniture,Tables,12995.2915,-3.272331e+02,-2.518090e+00,11,2.014595
5,SAFCO Arco Folding Chair,Furniture,Chairs,11572.7800,1.179374e+03,1.019093e+01,10,2.003386
6,"Howard Miller 13-3/4"" Diameter Brushed Chrome ...",Furniture,Furnishings,1438.1320,2.351407e+02,1.635043e+01,12,1.972961
7,Flat Face Poster Frame,Furniture,Furnishings,678.2400,1.209528e+02,1.783333e+01,12,1.961837
8,HON 5400 Series Task Chairs for Big and Tall,Furniture,Chairs,21870.5760,5.684342e-14,2.599082e-16,8,1.961743
9,DMI Eclipse Executive Suite Bookcases,Furniture,Bookcases,12921.6430,1.968640e+01,1.523521e-01,10,1.960367


In [20]:
# Display the bottom products based on the new adjusted composite score
worst_products_by_category = metrics_adjusted_combined.groupby('category').apply(lambda x: x.nsmallest(10, 'composite_score')).reset_index(drop=True)
worst_products_by_category[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'profit_margin', 'active_months', 'composite_score']]


,product_name,category,subcategory,total_sales,total_profit,profit_margin,active_months,composite_score
0,"Bush Westfield Collection Bookcases, Dark Cher...",Furniture,Bookcases,90.8820,-190.8522,-210.000000,1,0.456412
1,Eldon Executive Woodline II Cherry Finish Desk...,Furniture,Furnishings,98.1360,-78.5088,-80.000000,2,0.931255
2,BoxOffice By Design Rectangular and Half-Moon ...,Furniture,Tables,1706.2500,-1148.4375,-67.307692,3,1.036449
3,Riverside Furniture Stanwyck Manor Table Series,Furniture,Tables,917.9200,-415.9325,-45.312500,2,1.041393
4,"Bush Birmingham Collection Bookcase, Dark Cherry",Furniture,Bookcases,825.1740,-117.8820,-14.285714,1,1.072672
5,Anderson Hickey Conga Table Tops & Accessories,Furniture,Tables,51.7820,-17.5145,-33.823529,2,1.074374
6,"Bush Saratoga Collection 5-Shelf Bookcase, Han...",Furniture,Bookcases,119.8330,-12.6882,-10.588235,1,1.075685
7,"Atlantic Metals Mobile 2-Shelf Bookcases, Cust...",Furniture,Bookcases,400.0268,-113.2606,-28.313253,2,1.094172
8,"Tenex Traditional Chairmats for Hard Floors, A...",Furniture,Furnishings,180.4600,-46.4040,-25.714286,2,1.100566
9,Office Star - Contemporary Task Swivel chair w...,Furniture,Chairs,694.1940,-158.4858,-22.830189,2,1.114492


### Category Performance

In [21]:
# Grouping data by category and subcategory to calculate total sales, average profit margin, and total profit
category_performance = product_metrics.groupby(['category', 'subcategory']).agg(
    total_sales=('total_sales', 'sum'),
    average_profit_margin=('profit_margin', 'mean'),
    total_profit=('total_profit', 'sum')
).reset_index()

# Sorting the results by category and then by total sales within each category
category_performance.sort_values(by=['category', 'total_sales'], ascending=False, inplace=True)
category_performance


,category,subcategory,total_sales,average_profit_margin,total_profit
16,Technology,Phones,326628.7900,12.555032,43758.1530
15,Technology,Machines,186198.6310,-2.028366,1925.5569
13,Technology,Accessories,165795.4080,21.733481,41328.4531
14,Technology,Copiers,148628.0600,34.066789,55302.8354
11,Office Supplies,Storage,222667.5580,10.303619,21150.7772
6,Office Supplies,Binders,202984.2770,10.332846,30018.1745
4,Office Supplies,Appliances,107532.1610,9.271038,18138.0054
10,Office Supplies,Paper,77601.0980,42.778508,33634.1873
12,Office Supplies,Supplies,46189.0280,11.225482,-1200.6072
5,Office Supplies,Art,27014.6320,26.084450,6498.3676


### Insights

- **Technology**: Shows strong performance, particularly in Copiers, with high profit margins.
- **Office Supplies**: Show diverse performance; notably, Paper and Binders have high total sales and profit margins, making them lucrative categories.
- **Furniture**: Is mixed, with Chairs performing well in sales but low in profit margin, and Tables and Bookcases struggling with negative profit margins.


# Weighting Scheme:

## Sales Metrics (40% of total score)
- Total Sales: 20%
- Total Units Sold: 10%
- Sales Percentage of Total: 10%

## Pricing Metrics (10% of total score)
- Average Sale Price: 5%
- Min/Max Sale Price Range: 5%

## Profitability Metrics (30% of total score)
- Profit Margin: 15%
- Average Profit Per Unit: 15%

## Operational Metrics (10% of total score)
- Total Orders: 5%
- Active Months: 5%

## Impact of Discounts (10% of total score)
- Discount Impact on Sales: 5%
- Discount Impact on Profit: 5%


In [22]:
# Select relevant metrics
metrics = product_metrics[['total_sales', 'total_units_sold', 'sales_percentage_of_total',
                        'average_sale_price', 'min_sale_price_per_unit', 'max_sale_price_per_unit',
                        'profit_margin', 'average_profit_per_unit', 'total_orders', 'active_months',
                        'discount_impact_on_sales', 'discount_impact_on_profit']]

# Normalize the metrics
scaler = MinMaxScaler()
normalized_metrics = scaler.fit_transform(metrics)

# Define weights (as percentages)
weights = [0.20, 0.10, 0.10, 0.05, 0.025, 0.025, 0.15, 0.15, 0.05, 0.05, 0.05, 0.05]

# Calculate the composite score
composite_scores = normalized_metrics.dot(weights)
product_metrics['extended_composite_score'] = composite_scores

# Display the updated DataFrame with new composite scores
product_metrics[['product_id', 'composite_score', 'extended_composite_score']].head()

,product_id,composite_score,extended_composite_score
0,FUR-BO-10000112,1.072672,0.252619
1,FUR-BO-10000330,1.274534,0.275937
2,FUR-BO-10000362,1.480955,0.306398
3,FUR-BO-10000468,1.415097,0.290639
4,FUR-BO-10000711,1.281772,0.282309


In [23]:
# Group data by category and display the top products based on the new adjusted composite score
top_products_by_category = product_metrics.groupby('category').apply(lambda x: x.nlargest(10, 'extended_composite_score')).reset_index(drop=True)
top_products_by_category[['product_name', 'category', 'subcategory', 'total_sales', 'total_profit', 'profit_margin', 'active_months', 'composite_score', 'extended_composite_score']]

,product_name,category,subcategory,total_sales,total_profit,profit_margin,active_months,composite_score,extended_composite_score
0,HON 5400 Series Task Chairs for Big and Tall,Furniture,Chairs,21870.5760,5.684342e-14,2.599082e-16,8,1.961743,0.460810
1,SAFCO Arco Folding Chair,Furniture,Chairs,11572.7800,1.179374e+03,1.019093e+01,10,2.003386,0.434761
2,Bretford Rectangular Conference Table Tops,Furniture,Tables,12995.2915,-3.272331e+02,-2.518090e+00,11,2.014595,0.428745
3,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs,10637.5280,1.927442e+03,1.811927e+01,7,1.820263,0.419581
4,DMI Eclipse Executive Suite Bookcases,Furniture,Bookcases,12921.6430,1.968640e+01,1.523521e-01,10,1.960367,0.418142
5,Chromcraft Round Conference Tables,Furniture,Tables,8209.0590,-1.899761e+02,-2.314225e+00,11,1.941549,0.414432
6,Global Troy Executive Leather Low-Back Tilter,Furniture,Chairs,12975.3820,9.518620e+02,7.335907e+00,8,1.867839,0.408840
7,"Situations Contoured Folding Chairs, 4/Set",Furniture,Chairs,3499.3140,1.987440e+02,5.679513e+00,13,2.043949,0.401438
8,"Eldon Image Series Desk Accessories, Burgundy",Furniture,Furnishings,426.5600,1.167148e+02,2.736187e+01,15,2.201231,0.396670
9,"Global High-Back Leather Tilter, Burgundy",Furniture,Chairs,5436.1580,-4.747414e+02,-8.733032e+00,11,1.868427,0.391184


In [24]:
# Specify the path for the new CSV file
output_file_path = '../data/product_metrics.csv'

# Write the dataset to a new CSV file
product_metrics.to_csv(output_file_path, index=False)

output_file_path

'../data/product_metrics.csv'

In [25]:
product_metrics

,product_id,product_name,category,subcategory,total_sales,total_units_sold,average_sale_price,min_sale_price_per_unit,max_sale_price_per_unit,avg_sale_price_per_unit,total_profit,average_discount,average_processing_time,total_returns,total_loss,profit_margin,total_orders,average_profit_per_unit,sales_percentage_of_total,profit_rank,sales_rank,discount_impact_on_sales,discount_impact_on_profit,active_months,composite_score,extended_composite_score
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",Furniture,Bookcases,825.174,9,825.174000,91.686,91.686,91.686000,-117.8820,0.300000,2.000000,0,-117.8820,-14.285714,1,-13.098000,0.036223,1740.0,570.0,35.841523,-156.782664,1,1.072672,0.252619
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",Furniture,Bookcases,1064.624,10,354.874667,102.833,120.980,108.882000,24.1960,0.100000,1.666667,0,-9.6784,2.272727,3,2.419600,0.046735,1152.0,499.0,75.260182,-88.345012,3,1.274534,0.275937
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases,2154.348,14,430.869600,119.686,170.980,148.752600,311.1836,0.130000,4.000000,0,-35.9058,14.444444,5,22.227400,0.094571,268.0,283.0,254.652369,49.894247,5,1.480955,0.306398
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,Furniture,Bookcases,723.842,21,120.640333,14.574,48.580,39.673667,-133.1092,0.183333,3.666667,0,-195.2916,-18.389262,6,-6.338533,0.031775,1751.0,602.0,19.160080,-164.117464,6,1.415097,0.290639
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",Furniture,Bookcases,851.760,12,425.880000,70.980,70.980,70.980000,229.9752,0.000000,6.500000,0,0.0000,27.000000,2,19.164600,0.037391,337.0,560.0,40.218155,10.776916,2,1.281772,0.282309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,Technology,Phones,516.530,11,172.176667,43.960,54.950,47.623333,87.3705,0.133333,3.333333,0,0.0000,16.914894,3,7.942773,0.022675,639.0,723.0,-14.967968,-57.914442,3,1.312543,0.279644
1856,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,1085.238,19,217.047600,53.592,66.990,58.951200,130.6305,0.120000,4.000000,0,0.0000,12.037037,5,6.875289,0.047640,481.0,491.0,78.653694,-37.076502,5,1.450893,0.300439
1857,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",Technology,Phones,56.164,8,18.721333,5.912,7.390,6.897333,13.0064,0.066667,2.333333,0,0.0000,23.157895,3,1.625800,0.002465,1351.0,1603.0,-90.754188,-93.734938,3,1.322096,0.276023
1858,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,Technology,Phones,341.666,4,170.833000,80.392,100.490,90.441000,40.1960,0.100000,3.000000,0,0.0000,11.764706,2,10.049000,0.014998,974.0,882.0,-43.754372,-80.637960,2,1.221045,0.264626


In [26]:
# Check to see if any merging operations have introduced duplicate raws
product_metrics['product_id'].value_counts()

product_id
FUR-BO-10000112    1
OFF-PA-10003883    1
OFF-PA-10004071    1
OFF-PA-10004041    1
OFF-PA-10004040    1
                  ..
OFF-AR-10004269    1
OFF-AR-10004260    1
OFF-AR-10004165    1
OFF-AR-10004078    1
TEC-PH-10004977    1
Name: count, Length: 1860, dtype: int64